In [19]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
%config InlineBackend.figure_formats = ['retina']
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
info_patents = pd.read_pickle('../new_info_patents')
info_companies = pd.read_csv('./Data/new_info_companies.csv')

In [4]:
info_companies.drop(columns = [info_companies.columns.values[0]], inplace = True)

In [7]:
stat_columns = ["assignee_id", "assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors", "proportion_marked_patents", "proportion_multi_ipc", "proportion_A_ipc", "proportion_B_ipc", "proportion_C_ipc", "proportion_D_ipc", "proportion_E_ipc", "proportion_F_ipc", "proportion_G_ipc", "proportion_H_ipc"]
interest_info_companies = info_companies[stat_columns]

# OLS Regressions

### without entity fixed effects

In [17]:
reg_companies = interest_info_companies[["assignee_total_num_patents", "assignee_total_num_inventors", "proportion_marked_patents", "proportion_multi_ipc", "proportion_A_ipc", "proportion_B_ipc", "proportion_C_ipc", "proportion_D_ipc", "proportion_E_ipc", "proportion_F_ipc", "proportion_G_ipc", "proportion_H_ipc"]]
model = sm.OLS(reg_companies.proportion_marked_patents, reg_companies.drop('proportion_marked_patents', axis = 1))
result = model.fit(cov_type = 'hc0')
print(result.summary())

                                OLS Regression Results                               
Dep. Variable:     proportion_marked_patents   R-squared:                       0.110
Model:                                   OLS   Adj. R-squared:                  0.104
Method:                        Least Squares   F-statistic:                       nan
Date:                       Mon, 31 May 2021   Prob (F-statistic):                nan
Time:                               11:16:59   Log-Likelihood:                -14.383
No. Observations:                       1695   AIC:                             50.77
Df Residuals:                           1684   BIC:                             110.6
Df Model:                                 10                                         
Covariance Type:                         hc0                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------

In [20]:
vif = pd.DataFrame()
vif["variables"] = reg_companies.columns
vif["VIF"] = [variance_inflation_factor(reg_companies.values, i) for i in range(reg_companies.shape[1])]
vif

,variables,VIF
0,assignee_total_num_patents,8.318907
1,assignee_total_num_inventors,8.425058
2,proportion_marked_patents,1.123104
3,proportion_multi_ipc,1.126157
4,proportion_A_ipc,1.461959
5,proportion_B_ipc,1.381038
6,proportion_C_ipc,1.500389
7,proportion_D_ipc,1.075185
8,proportion_E_ipc,1.077322
9,proportion_F_ipc,1.192314


count      1695.000000
mean        849.617109
std        4732.999739
min           4.000000
25%          12.000000
50%          39.000000
75%         160.000000
max      118728.000000
Name: assignee_total_num_patents, dtype: float64